In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import glob
import os
import scipy.stats
from matplotlib import cm
import seaborn as sns
import pickle

In [5]:
jetdic = pickle.load(open( "../JET/JET_OUT/jettrack_2D.p", "rb" ))

In [6]:
weights = np.cos(np.deg2rad(jetdic['NOAA'].lat)) #area weighted

In [8]:
#mean absolute error calc
results=[]
for reference in jetdic:
    for index in jetdic:
        MAE={}
        for season in ['DJF','MAM','JJA','SON']:
            ref = jetdic[reference].where(jetdic[reference].time.dt.season==season).mean(dim='time')
            x = jetdic[index].where(jetdic[index].time.dt.season==season).mean(dim='time')
            MAE[season] = (np.abs(ref - x)).weighted(weights).sum(('lat','lon'))
        results.append([reference, index,np.mean(MAE['DJF'].values + MAE['MAM'].values + MAE['JJA'].values + MAE['SON'].values)])

In [10]:
models = jetdic.keys()

In [11]:
df = pd.DataFrame(models,columns = ['models'])
start = 0
end = len(models)
for index in models:
    df[index] = np.array(results[start:end])[:,-1].astype(float)
    start = start + len(models)
    end = end + len(models)

In [12]:
df = df.set_index('models')

df.to_csv('Jet_2D_Linkage.csv')